# Covid-19 Data Visualisation

In this notebook I am going to do a number of things:
1. Scrap covid 19 data from NHS website
2. Clean the data and separate it into dataframes
3. Plot the data using bokeh

***
## Data Acquisition
Here I am going to scrap the covid-19 data from the NHS England website.

NHS England publish their daily figures for deaths at the following site: https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-daily-deaths/

#### Notes on the data
From the website

"This section contains information on deaths of patients who have died in hospitals in England and had tested positive for COVID-19 at time of death. All deaths are recorded against the date of death rather than the date the deaths were announced. Interpretation of the figures should take into account the fact that totals by date of death, particularly for most recent days, are likely to be updated in future releases. For example as deaths are confirmed as testing positive for COVID-19, as more post-mortem tests are processed and data from them are validated. Any changes are made clear in the daily files.

These figures will be updated at 2pm each day and include confirmed cases reported at 5pm the previous day. Confirmation of COVID-19 diagnosis, death notification and reporting in central figures can take up to several days and the hospitals providing the data are under significant operational pressure. This means that the totals reported at 5pm on each day may not include all deaths that occurred on that day or on recent prior days.

These figures do not include deaths outside hospital, such as those in care homes. This approach makes it possible to compile deaths data on a daily basis using up to date figures."

Essentially, every day they post a new updated xlsx file with the up to date figures they have. So the figures from the previous few days are likely to change as some people die and only are confirmed to of had covid-19 post-mortem. Therefore, the most recent data release should be used.

### Import Libraries

In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import bokeh
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

### Web Scraping

This is the url to the page on the NHS england website that publishes the daily covid-19 deaths

In [3]:
url = "https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-daily-deaths/"

If the response == 200 then the requestw as successful

In [4]:
response = requests.get(url)
response

<Response [200]>

Now use BeautifulSoup to parse the text data

response.text returns a huge string of the html code contained on the url's page.

In [5]:
response.text

'<!DOCTYPE HTML>\n<!--[if lt IE 7]><html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-US"> <![endif]-->\n<!--[if IE 7]><html class="no-js lt-ie9 lt-ie8" lang="en-US"> <![endif]-->\n<!--[if IE 8]><html class="no-js lt-ie9" lang="en-US"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->\n<head>\n    <title>Statistics &raquo; COVID-19 Daily Deaths</title>\n    <script type="text/javascript">\n        var addthis_config =\n        {\n            ui_tabindex:0\n        }\n    </script>\n    <meta charset="utf-8">\n    <meta http-equiv="cleartype" content="on">\n    <meta name="description" content="Health and high quality care for all,  &lt;br /&gt;now and for future generations">\n    <meta name="author" content="Statistics">\n\n    <link rel="alternate" type="application/rss+xml" title="RSS2.0" href="https://www.england.nhs.uk/statistics/feed/" />\n\n    <link rel=\'stylesheet\' id=\'wp-block-library-css\'  href=\'https://www.england.nhs.uk/statistics/

In [6]:
soup = BeautifulSoup(response.text, "html.parser")

Use findAll() to find all the 'a' tags in the html code. These tags represent hyperlinks in html

In [7]:
all_hplinks = soup.find_all('a')

for i, hplink in enumerate(all_hplinks):
    if str(date.today()) in hplink:
        print(i, hplink)
        print()

By going to the web page and inspecting it the link we want is

"https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-14-April-2020.xlsx"

It is at location 423 of the resulting array so lets get that and asisgn it to the variable total_deaths_html

In [8]:
total_deaths_html = soup.find_all('a')[423]
total_deaths_html

<a href="https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-14-April-2020.xlsx">COVID 19 total announced deaths 14 April 2020</a>

Finally lets strip off the html tag so that we are left with the actual link as a string

In [9]:
total_deaths_link = total_deaths_html['href']
total_deaths_link

'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-14-April-2020.xlsx'

Now let's download that file to my local machine. Also name the file we download with today's date

In [10]:
filename = 'data/COVID-19-total-announced-deaths-' + str(date.today()) + ".xlsx"
urllib.request.urlretrieve(total_deaths_link, filename)

('data/COVID-19-total-announced-deaths-2020-04-14.xlsx',
 <http.client.HTTPMessage at 0x125684f90>)

In [11]:
# Needed if downloading many files to avoid being flagged as a spammer. Not needed here but if using a for loop
# to get many files then it is needed
time.sleep(1)

***
## Data Cleaning
Before plotting the data we need to get it into a clean DataFrame. First step is to load the raw data into a unsorted raw DataFramme

In [3]:
raw_data_trusts = pd.read_excel("./data/COVID-19-total-announced-deaths-2020-04-14.xlsx", sheet_name=0, header=15, )
raw_data_region = pd.read_excel("./data/COVID-19-total-announced-deaths-2020-04-14.xlsx", sheet_name=1, header=15)
raw_data_age = pd.read_excel("./data/COVID-19-total-announced-deaths-2020-04-14.xlsx", sheet_name=2, header=15)

From looking at the formatting of the excel file the Unnamed columns represent a balnk column so we can just drop those

In [4]:
raw_data_trusts.describe()

,Unnamed: 0,Unnamed: 2,Up to 01-Mar-20,2020-03-01 00:00:00,2020-03-02 00:00:00,2020-03-03 00:00:00,2020-03-04 00:00:00,2020-03-05 00:00:00,2020-03-06 00:00:00,2020-03-07 00:00:00,...,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68
count,0.0,0.0,183.000000,183.000000,183.0,183.000000,183.000000,183.000000,183.000000,183.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,NaN,NaN,0.010929,0.010929,0.0,0.054645,0.010929,0.043716,0.032787,0.010929,...,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0
std,NaN,NaN,0.104254,0.104254,0.0,0.415718,0.104254,0.328586,0.254663,0.104254,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0
25%,NaN,NaN,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0
50%,NaN,NaN,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0
75%,NaN,NaN,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0
max,NaN,NaN,1.000000,1.000000,0.0,5.000000,1.000000,4.000000,3.000000,1.000000,...,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0


In [5]:
raw_data_trusts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 69 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             0 non-null      float64
 1   NHS England Region     183 non-null    object 
 2   Unnamed: 2             0 non-null      float64
 3   Code                   183 non-null    object 
 4   Name                   183 non-null    object 
 5   Up to 01-Mar-20        183 non-null    float64
 6   2020-03-01 00:00:00    183 non-null    float64
 7   2020-03-02 00:00:00    183 non-null    float64
 8   2020-03-03 00:00:00    183 non-null    float64
 9   2020-03-04 00:00:00    183 non-null    float64
 10  2020-03-05 00:00:00    183 non-null    float64
 11  2020-03-06 00:00:00    183 non-null    float64
 12  2020-03-07 00:00:00    183 non-null    float64
 13  2020-03-08 00:00:00    183 non-null    float64
 14  2020-03-09 00:00:00    183 non-null    float64
 15  2020-0

Drop the blank columns and look at the remaining data. There is still a row of NaN values which we need to drop

In [6]:
for col in raw_data_trusts.columns:
    if (type(col) == str) and ("Unnamed" in col):
        raw_data_trusts.drop(col, axis=1, inplace=True)
        
raw_data_trusts.head()

,NHS England Region,Code,Name,Up to 01-Mar-20,2020-03-01 00:00:00,2020-03-02 00:00:00,2020-03-03 00:00:00,2020-03-04 00:00:00,2020-03-05 00:00:00,2020-03-06 00:00:00,...,2020-04-06 00:00:00,2020-04-07 00:00:00,2020-04-08 00:00:00,2020-04-09 00:00:00,2020-04-10 00:00:00,2020-04-11 00:00:00,2020-04-12 00:00:00,2020-04-13 00:00:00,Awaiting verification,Total
0,-,-,ENGLAND,1.0,1.0,0.0,5.0,1.0,4.0,3.0,...,631.0,692.0,750.0,636.0,579.0,575.0,437.0,122.0,1.0,11005.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,East Of England,NQ1,ANGLIAN COMMUNITY ENTERPRISE CIC - CLACTON HOS...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,East Of England,NQ1,ANGLIAN COMMUNITY ENTERPRISE CIC - FRYATT HOSP...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,5.0
4,East Of England,RDD,BASILDON AND THURROCK UNIVERSITY HOSPITALS NHS...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,8.0,7.0,10.0,9.0,10.0,6.0,9.0,0.0,0.0,131.0


In [7]:
raw_data_trusts.columns

Index([   'NHS England Region',                  'Code',
                        'Name',       'Up to 01-Mar-20',
           2020-03-01 00:00:00,     2020-03-02 00:00:00,
           2020-03-03 00:00:00,     2020-03-04 00:00:00,
           2020-03-05 00:00:00,     2020-03-06 00:00:00,
           2020-03-07 00:00:00,     2020-03-08 00:00:00,
           2020-03-09 00:00:00,     2020-03-10 00:00:00,
           2020-03-11 00:00:00,     2020-03-12 00:00:00,
           2020-03-13 00:00:00,     2020-03-14 00:00:00,
           2020-03-15 00:00:00,     2020-03-16 00:00:00,
           2020-03-17 00:00:00,     2020-03-18 00:00:00,
           2020-03-19 00:00:00,     2020-03-20 00:00:00,
           2020-03-21 00:00:00,     2020-03-22 00:00:00,
           2020-03-23 00:00:00,     2020-03-24 00:00:00,
           2020-03-25 00:00:00,     2020-03-26 00:00:00,
           2020-03-27 00:00:00,     2020-03-28 00:00:00,
           2020-03-29 00:00:00,     2020-03-30 00:00:00,
           2020-03-31 00:00:00,

Dropping the row of NaN values.
Also drop the first row as it is just an aggregation of the other rows.
I will also drop the 'Awaiting verification' column as it doesn't really add anything

In [8]:
raw_data_trusts.dropna(inplace=True)
raw_data_trusts.drop(index=0, inplace=True, errors='ignore')
raw_data_trusts.drop('Awaiting verification', axis=1, inplace=True, errors='ignore')
raw_data_trusts.head()

,NHS England Region,Code,Name,Up to 01-Mar-20,2020-03-01 00:00:00,2020-03-02 00:00:00,2020-03-03 00:00:00,2020-03-04 00:00:00,2020-03-05 00:00:00,2020-03-06 00:00:00,...,2020-04-05 00:00:00,2020-04-06 00:00:00,2020-04-07 00:00:00,2020-04-08 00:00:00,2020-04-09 00:00:00,2020-04-10 00:00:00,2020-04-11 00:00:00,2020-04-12 00:00:00,2020-04-13 00:00:00,Total
2,East Of England,NQ1,ANGLIAN COMMUNITY ENTERPRISE CIC - CLACTON HOS...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,East Of England,NQ1,ANGLIAN COMMUNITY ENTERPRISE CIC - FRYATT HOSP...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,5.0
4,East Of England,RDD,BASILDON AND THURROCK UNIVERSITY HOSPITALS NHS...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,8.0,8.0,7.0,10.0,9.0,10.0,6.0,9.0,0.0,131.0
5,East Of England,RC1,BEDFORD HOSPITAL NHS TRUST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,2.0,2.0,8.0,2.0,2.0,5.0,1.0,0.0,52.0
6,East Of England,RGT,CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,3.0,3.0,3.0,4.0,0.0,0.0,32.0


This looks good now. We have 183 entries in each column and 183 non-null objects.

In [9]:
raw_data_trusts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 2 to 183
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NHS England Region   182 non-null    object 
 1   Code                 182 non-null    object 
 2   Name                 182 non-null    object 
 3   Up to 01-Mar-20      182 non-null    float64
 4   2020-03-01 00:00:00  182 non-null    float64
 5   2020-03-02 00:00:00  182 non-null    float64
 6   2020-03-03 00:00:00  182 non-null    float64
 7   2020-03-04 00:00:00  182 non-null    float64
 8   2020-03-05 00:00:00  182 non-null    float64
 9   2020-03-06 00:00:00  182 non-null    float64
 10  2020-03-07 00:00:00  182 non-null    float64
 11  2020-03-08 00:00:00  182 non-null    float64
 12  2020-03-09 00:00:00  182 non-null    float64
 13  2020-03-10 00:00:00  182 non-null    float64
 14  2020-03-11 00:00:00  182 non-null    float64
 15  2020-03-12 00:00:00  182 non-null    flo

In [10]:
raw_data_trusts.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Up to 01-Mar-20,182.0,0.005495,0.074125,0.0,0.0,0.0,0.00,1.0
2020-03-01 00:00:00,182.0,0.005495,0.074125,0.0,0.0,0.0,0.00,1.0
2020-03-02 00:00:00,182.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
2020-03-03 00:00:00,182.0,0.027473,0.194718,0.0,0.0,0.0,0.00,2.0
2020-03-04 00:00:00,182.0,0.005495,0.074125,0.0,0.0,0.0,0.00,1.0
2020-03-05 00:00:00,182.0,0.021978,0.147016,0.0,0.0,0.0,0.00,1.0
2020-03-06 00:00:00,182.0,0.016484,0.127677,0.0,0.0,0.0,0.00,1.0
2020-03-07 00:00:00,182.0,0.005495,0.074125,0.0,0.0,0.0,0.00,1.0
2020-03-08 00:00:00,182.0,0.027473,0.194718,0.0,0.0,0.0,0.00,2.0
2020-03-09 00:00:00,182.0,0.010989,0.104539,0.0,0.0,0.0,0.00,1.0


Now let's clean the other two datasets

In [11]:
for dataset in [raw_data_region, raw_data_age]:
    for col in dataset.columns:
        if (type(col) == str) and ("Unnamed" in col):
            dataset.drop(col, axis=1, inplace=True, errors='ignore')
    dataset.dropna(inplace=True)
    dataset.drop(index=0, inplace=True, errors='ignore')
    dataset.drop('Awaiting verification', inplace=True, axis=1, errors='ignore')

In [12]:
raw_data_region.head().transpose()

,2,3,4,5,6
NHS England Region,East Of England,London,Midlands,North East And Yorkshire,North West
Up to 01-Mar-20,0,0,1,0,0
2020-03-01 00:00:00,0,0,1,0,0
2020-03-02 00:00:00,0,0,0,0,0
2020-03-03 00:00:00,3,0,1,0,0
2020-03-04 00:00:00,1,0,0,0,0
2020-03-05 00:00:00,0,1,0,0,1
2020-03-06 00:00:00,1,1,0,0,0
2020-03-07 00:00:00,0,1,0,0,0
2020-03-08 00:00:00,0,0,3,0,1


In [13]:
raw_data_age.head().transpose()

,2,3,4,5,6
Age group,0 - 19 yrs,20 - 39,40 - 59,60 - 79,80+
Up to 01-Mar-20,0,0,0,0,1
2020-03-01 00:00:00,0,0,0,0,1
2020-03-02 00:00:00,0,0,0,0,0
2020-03-03 00:00:00,0,0,1,1,3
2020-03-04 00:00:00,0,0,0,0,1
2020-03-05 00:00:00,0,0,1,2,1
2020-03-06 00:00:00,0,0,0,1,2
2020-03-07 00:00:00,0,0,0,0,1
2020-03-08 00:00:00,0,0,0,4,1


In [14]:
raw_data_region.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 2 to 8
Data columns (total 47 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NHS England Region   7 non-null      object 
 1   Up to 01-Mar-20      7 non-null      float64
 2   2020-03-01 00:00:00  7 non-null      float64
 3   2020-03-02 00:00:00  7 non-null      float64
 4   2020-03-03 00:00:00  7 non-null      float64
 5   2020-03-04 00:00:00  7 non-null      float64
 6   2020-03-05 00:00:00  7 non-null      float64
 7   2020-03-06 00:00:00  7 non-null      float64
 8   2020-03-07 00:00:00  7 non-null      float64
 9   2020-03-08 00:00:00  7 non-null      float64
 10  2020-03-09 00:00:00  7 non-null      float64
 11  2020-03-10 00:00:00  7 non-null      float64
 12  2020-03-11 00:00:00  7 non-null      float64
 13  2020-03-12 00:00:00  7 non-null      float64
 14  2020-03-13 00:00:00  7 non-null      float64
 15  2020-03-14 00:00:00  7 non-null      float64

In [15]:
raw_data_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 2 to 6
Data columns (total 47 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age group            5 non-null      object 
 1   Up to 01-Mar-20      5 non-null      float64
 2   2020-03-01 00:00:00  5 non-null      float64
 3   2020-03-02 00:00:00  5 non-null      float64
 4   2020-03-03 00:00:00  5 non-null      float64
 5   2020-03-04 00:00:00  5 non-null      float64
 6   2020-03-05 00:00:00  5 non-null      float64
 7   2020-03-06 00:00:00  5 non-null      float64
 8   2020-03-07 00:00:00  5 non-null      float64
 9   2020-03-08 00:00:00  5 non-null      float64
 10  2020-03-09 00:00:00  5 non-null      float64
 11  2020-03-10 00:00:00  5 non-null      float64
 12  2020-03-11 00:00:00  5 non-null      float64
 13  2020-03-12 00:00:00  5 non-null      float64
 14  2020-03-13 00:00:00  5 non-null      float64
 15  2020-03-14 00:00:00  5 non-null      float64

Finally let's assign these cleaned DataFrames to a new variables

In [16]:
data_trusts = raw_data_trusts
data_region = raw_data_region
data_age = raw_data_age

***
## Feature Engineering
I want to make a geographical plot later with the trusts dataset but they do not have addresses. I managed to find a csv file at https://data.england.nhs.uk/dataset/ods-nhs-trusts-and-sites that lists the current and closed NHS Trusts including code, name, address, etc etc.

The trusts data already had a code attached to ecah entry so hopefully this is the same as the code in this dataset which would make matching them easy.

First load the csv containing the additional info on the trusts

In [17]:
data_trusts_additional = pd.read_csv("./data/etr.csv", header=None)

In [18]:
data_trusts_additional

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,R1A,WORCESTERSHIRE HEALTH AND CARE NHS TRUST,Y55,Q77,ISAAC MADDOX HOUSE,SHRUB HILL INDUSTRIAL ESTATE,NaN,WORCESTER,WORCESTERSHIRE,WR4 9RW,...,NaN,NaN,NaN,NaN,0,NaN,F,NaN,NaN,NaN
1,R1C,SOLENT NHS TRUST,Y57,Q70,SOLENT NHS TRUST HEADQUARTERS,HIGHPOINT VENUE,BURSLEDON ROAD,SOUTHAMPTON,HAMPSHIRE,SO19 8BR,...,NaN,NaN,NaN,NaN,0,NaN,J,NaN,NaN,NaN
2,R1D,SHROPSHIRE COMMUNITY HEALTH NHS TRUST,Y55,Q76,WILLIAM FARR HOUSE,MYTTON OAK ROAD,NaN,SHREWSBURY,SHROPSHIRE,SY3 8XL,...,NaN,NaN,NaN,NaN,0,NaN,E,NaN,NaN,NaN
3,R1E,STAFFORDSHIRE AND STOKE ON TRENT PARTNERSHIP N...,Y55,Q76,"2ND FLOOR, MORSTON HOUSE",NaN,NaN,NEWCASTLE-UNDER-LYME,STAFFORDSHIRE,ST5 1QG,...,NaN,NaN,NaN,NaN,0,NaN,E,NaN,NaN,NaN
4,R1F,ISLE OF WIGHT NHS TRUST,Y57,Q70,ST MARY'S HOSPITAL,PARKHURST ROAD,NaN,NEWPORT,ISLE OF WIGHT,PO30 5TG,...,NaN,NaN,NaN,NaN,0,NaN,J,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,RYT,PUBLIC HEALTH WALES NHS TRUST,W00,Q99,14 CATHEDRAL ROAD,NaN,NaN,CARDIFF,SOUTH GLAMORGAN,CF11 9LJ,...,NaN,NaN,NaN,NaN,0,NaN,W,NaN,NaN,NaN
235,RYV,CAMBRIDGESHIRE COMMUNITY SERVICES NHS TRUST,Y55,Q79,UNIT 3,MEADOW LANE,NaN,ST. IVES,CAMBRIDGESHIRE,PE27 4LG,...,NaN,NaN,NaN,NaN,0,NaN,G,NaN,NaN,NaN
236,RYW,BIRMINGHAM COMMUNITY HEALTHCARE NHS FOUNDATION...,Y55,Q77,"3, PRIESTLEY WHARF","HOLT STREET,","BIRMINGHAM SCIENCE PARK, ASTON",BIRMINGHAM,WEST MIDLANDS,B7 4BN,...,NaN,NaN,NaN,NaN,1,NaN,F,NaN,NaN,NaN
237,RYX,CENTRAL LONDON COMMUNITY HEALTHCARE NHS TRUST,Y56,Q71,7TH FLOOR,64 VICTORIA STREET,NaN,LONDON,GREATER LONDON,SW1E 6QP,...,NaN,NaN,NaN,NaN,0,NaN,H,NaN,NaN,NaN


From https://nhsenglandfilestore.s3.amazonaws.com/ods/etr.pdf I was able to find out what the columns are so lets input those

In [19]:
columns = 'Organisation Code, Name, National_Grouping, High_Level_Health_Geography, Address_Line_1, Address_Line_2, \
Address_Line_3, Address_Line_4, Address_Line_5, Postcode, Open_Date, Close_Date, Null, Null, Null, Null, Null, Telephone, \
Null, null, Null, Amended_Record_Indicator, Null, GOR_Code, Null, Null, Null'.split(", ")
columns

['Organisation Code',
 'Name',
 'National_Grouping',
 'High_Level_Health_Geography',
 'Address_Line_1',
 'Address_Line_2',
 'Address_Line_3',
 'Address_Line_4',
 'Address_Line_5',
 'Postcode',
 'Open_Date',
 'Close_Date',
 'Null',
 'Null',
 'Null',
 'Null',
 'Null',
 'Telephone',
 'Null',
 'null',
 'Null',
 'Amended_Record_Indicator',
 'Null',
 'GOR_Code',
 'Null',
 'Null',
 'Null']

Rename columns to the columns I defined above

In [20]:
data_trusts_additional.columns = columns
data_trusts_additional.head()

,Organisation Code,Name,National_Grouping,High_Level_Health_Geography,Address_Line_1,Address_Line_2,Address_Line_3,Address_Line_4,Address_Line_5,Postcode,...,Telephone,Null,null,Null,Amended_Record_Indicator,Null,GOR_Code,Null,Null,Null
0,R1A,WORCESTERSHIRE HEALTH AND CARE NHS TRUST,Y55,Q77,ISAAC MADDOX HOUSE,SHRUB HILL INDUSTRIAL ESTATE,NaN,WORCESTER,WORCESTERSHIRE,WR4 9RW,...,NaN,NaN,NaN,NaN,0,NaN,F,NaN,NaN,NaN
1,R1C,SOLENT NHS TRUST,Y57,Q70,SOLENT NHS TRUST HEADQUARTERS,HIGHPOINT VENUE,BURSLEDON ROAD,SOUTHAMPTON,HAMPSHIRE,SO19 8BR,...,NaN,NaN,NaN,NaN,0,NaN,J,NaN,NaN,NaN
2,R1D,SHROPSHIRE COMMUNITY HEALTH NHS TRUST,Y55,Q76,WILLIAM FARR HOUSE,MYTTON OAK ROAD,NaN,SHREWSBURY,SHROPSHIRE,SY3 8XL,...,NaN,NaN,NaN,NaN,0,NaN,E,NaN,NaN,NaN
3,R1E,STAFFORDSHIRE AND STOKE ON TRENT PARTNERSHIP N...,Y55,Q76,"2ND FLOOR, MORSTON HOUSE",NaN,NaN,NEWCASTLE-UNDER-LYME,STAFFORDSHIRE,ST5 1QG,...,NaN,NaN,NaN,NaN,0,NaN,E,NaN,NaN,NaN
4,R1F,ISLE OF WIGHT NHS TRUST,Y57,Q70,ST MARY'S HOSPITAL,PARKHURST ROAD,NaN,NEWPORT,ISLE OF WIGHT,PO30 5TG,...,NaN,NaN,NaN,NaN,0,NaN,J,NaN,NaN,NaN


Let's see if we can match these records up using the 'Code' and 'Organisation Code' columns.

They appear to be the same

In [21]:
data_trusts['Code'].unique()

array(['NQ1', 'RDD', 'RC1', 'RGT', 'RT1', 'RWH', 'NAX', 'RDE', 'R1L',
       'RY4', 'RWR', 'RGP', 'RC9', 'RQ8', 'RD8', 'RM1', 'RY3', 'RGN',
       'RGM', 'RAJ', 'RQW', 'RCX', 'RWG', 'RGR', 'RF4', 'R1H', 'RV3',
       'RYX', 'RQM', 'RJ6', 'RWK', 'RVR', 'RJ1', 'NW6', 'RQX', 'RYJ',
       'RJZ', 'RAX', 'RJ2', 'R1K', 'RAT', 'RAP', 'RPG', 'RT3', 'RAL',
       'NN1', 'RV5', 'RQY', 'RJ7', 'RAS', 'RPY', 'RRV', 'RKL', 'RKE',
       'RYW', 'RFS', 'RY8', 'RXM', 'RLT', 'RNQ', 'RT5', 'RLY', 'RNS',
       'RP1', 'RHA', 'RX1', 'RXK', 'RK5', 'RXW', 'RJC', 'RNA', 'RL1',
       'RRJ', 'RL4', 'RWD', 'RRK', 'RKB', 'RTG', 'RWE', 'RJE', 'RBK',
       'RWP', 'R1A', 'RLQ', 'RCF', 'RFF', 'TAD', 'RAE', 'RWY', 'RXP',
       'RX4', 'RP5', 'RR7', 'RCD', 'RWA', 'RV9', 'RR8', 'RXF', 'RNN',
       'RVW', 'RJL', 'RTF', 'RXE', 'TAH', 'RHQ', 'RTR', 'R0B', 'RX3',
       'RTD', 'RFR', 'RCB', 'RXL', 'RMC', 'RXA', 'RJR', 'RJN', 'RXR',
       'RXN', 'RBQ', 'REM', 'R0A', 'RBT', 'RW6', 'RM3', 'RVY', 'RBN',
       'RWJ', 'RMP',

In [22]:
data_trusts_additional['Organisation Code'].unique()

array(['R1A', 'R1C', 'R1D', 'R1E', 'R1F', 'R1G', 'R1H', 'R1J', 'R1K',
       'RA2', 'RA3', 'RA4', 'RA7', 'RA9', 'RAE', 'RAJ', 'RAL', 'RAN',
       'RAP', 'RAS', 'RAT', 'RAX', 'RBA', 'RBD', 'RBK', 'RBL', 'RBN',
       'RBQ', 'RBS', 'RBT', 'RBV', 'RBZ', 'RC1', 'RC9', 'RCB', 'RCD',
       'RCF', 'RCU', 'RCX', 'RD1', 'RD3', 'RD8', 'RDD', 'RDE', 'RDR',
       'RDU', 'RDY', 'RDZ', 'RE9', 'REF', 'REM', 'REN', 'REP', 'RET',
       'RF4', 'RFF', 'RFR', 'RFS', 'RFW', 'RGD', 'RGM', 'RGN', 'RGP',
       'RGQ', 'RGR', 'RGT', 'RH5', 'RH8', 'RHA', 'RHM', 'RHQ', 'RHU',
       'RHW', 'RJ1', 'RJ2', 'RJ6', 'RJ7', 'RJ8', 'RJC', 'RJD', 'RJE',
       'RJF', 'RJL', 'RJN', 'RJR', 'RJX', 'RJZ', 'RK5', 'RK9', 'RKB',
       'RKE', 'RKL', 'RL1', 'RL4', 'RLN', 'RLQ', 'RLT', 'RLU', 'RLY',
       'RM1', 'RM2', 'RM3', 'RMC', 'RMP', 'RMY', 'RN3', 'RN5', 'RN7',
       'RNA', 'RNK', 'RNL', 'RNN', 'RNQ', 'RNS', 'RNU', 'RNZ', 'RP1',
       'RP4', 'RP5', 'RP6', 'RP7', 'RPA', 'RPC', 'RPG', 'RPY', 'RQ3',
       'RQ6', 'RQ8',

In [23]:
data_trusts.head()

,NHS England Region,Code,Name,Up to 01-Mar-20,2020-03-01 00:00:00,2020-03-02 00:00:00,2020-03-03 00:00:00,2020-03-04 00:00:00,2020-03-05 00:00:00,2020-03-06 00:00:00,...,2020-04-05 00:00:00,2020-04-06 00:00:00,2020-04-07 00:00:00,2020-04-08 00:00:00,2020-04-09 00:00:00,2020-04-10 00:00:00,2020-04-11 00:00:00,2020-04-12 00:00:00,2020-04-13 00:00:00,Total
2,East Of England,NQ1,ANGLIAN COMMUNITY ENTERPRISE CIC - CLACTON HOS...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,East Of England,NQ1,ANGLIAN COMMUNITY ENTERPRISE CIC - FRYATT HOSP...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,5.0
4,East Of England,RDD,BASILDON AND THURROCK UNIVERSITY HOSPITALS NHS...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,8.0,8.0,7.0,10.0,9.0,10.0,6.0,9.0,0.0,131.0
5,East Of England,RC1,BEDFORD HOSPITAL NHS TRUST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,2.0,2.0,8.0,2.0,2.0,5.0,1.0,0.0,52.0
6,East Of England,RGT,CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,3.0,3.0,3.0,4.0,0.0,0.0,32.0


In [36]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

In [80]:
address = ""
address_array = data_trusts_additional.iloc[1][['Address_Line_1',
 'Address_Line_2',
 'Address_Line_3',
 'Address_Line_4',
 'Address_Line_5',
 'Postcode']]

for i in address_array:
    if type(i) == str:
        address += i + ", "
    


In [81]:
address

'SOLENT NHS TRUST HEADQUARTERS, HIGHPOINT VENUE, BURSLEDON ROAD, SOUTHAMPTON, HAMPSHIRE, SO19 8BR, '

In [88]:
location = geolocator.geocode("SO19 8BR")

In [89]:
print(location.address)

Bitterne, Southampton, South East, England, SO19 8BR, United Kingdom


In [90]:
print((location.latitude, location.longitude))

(50.901687489735544, -1.3308598482947636)


***
***
## Plotting data
Now start to plot all of the data
***
### Age
I will start off with the age data

In [ ]:
data_age.head()

This is the total amount of deaths in each age group up until today

In [ ]:
plt.figure(figsize=(12,8))
plt.grid(which='both')
sns.barplot(x='Age group', y='Total', data=data_age)

Now lets plot a cumulative sum. First let's make a new DataFrame with the cumulative sum data in it

In [ ]:
age_groups = data_age['Age group'].unique()
df_age_groups = data_age.drop('Total', axis=1)
x = range(0,data_age.shape[1]-3)
new_index = [x.date() for x in data_age.columns[2:-1]]
df_cumulative = pd.DataFrame(index=new_index)
for age_g in age_groups:
    df_cumulative[age_g] = np.cumsum(df_age_groups[df_age_groups['Age group'] == age_g].values[0][2:], axis=0)
    

In [ ]:
df_cumulative.head()

In [ ]:
plt.figure(figsize=(12,8))
age_groups = data_age['Age group'].unique()
df_age_groups = data_age.drop('Total', axis=1)
x = data_age.columns[2:-1]
for age_g in age_groups:
    plt.plot(df_cumulative[age_g], label=age_g)

plt.xlabel("Date")
plt.ylabel("Total cumulative deaths")
plt.grid()
plt.legend()

#### Bokeh Interactive Plot
Lets plot the above data but on an iteractive plot

In [ ]:
from bokeh.plotting import ColumnDataSource

In [ ]:
src = ColumnDataSource(df_cumulative)
src.data

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool

colors = ['red', 'green', 'blue', 'grey', 'purple']
# output to static HTML file
output_file("age_group_cumulative.html")


# create a new plot with a title and axis labels
p = figure(title="Total deaths from Covid-19 in UK grouped by age", x_axis_label='Date',
           y_axis_label='Total deaths', x_axis_type="datetime")

for i, age_g in enumerate(age_groups):
    src = ColumnDataSource({'x':df_cumulative.index,
                           'y':df_cumulative[age_g]})
    p.line(x='x', y='y', source=src, line_width=1, line_color=colors[i], legend=age_g)
    p.circle(x='x', y='y', source=src, fill_color=colors[i], line_color=colors[i], size=6, legend=age_g)
    
    hover = HoverTool(tooltips =[
     ('age group',age_g),('(Date, Total deaths)','(@x{%F}, @y)')],
                     formatters={'x': 'datetime'})
                     
    p.add_tools(hover)

p.legend.location = "top_left"
show(p)

***
### Regions
Now let's make some plots for the region data

First rearrange the dataframe so that the index is the date

In [ ]:
regions = data_region['NHS England Region'].unique()

In [ ]:
data_region[data_region["NHS England Region"]==regions[0]]

In [ ]:
df_regions = pd.DataFrame(index=[x.date() for x in data_region.columns[2:-1]])

In [ ]:
regions = data_region['NHS England Region'].unique()
df_regions = data_region.drop('Total', axis=1)
x = range(0,data_region.shape[1]-3)
new_index = [x.date() for x in data_region.columns[2:-1]]
df_cumulative_region = pd.DataFrame(index=new_index)
for region in regions:
    df_cumulative_region[region] = np.cumsum(df_regions[df_regions['NHS England Region'] == region].values[0][2:], axis=0)

In [ ]:
df_cumulative_region.head()

In [ ]:
plt.figure(figsize=(12,8))
for region in regions:
    plt.plot(df_cumulative_region[region], label=region)

plt.xlabel("Date")
plt.ylabel("Total cumulative deaths")
plt.title("The total cumulative deaths grouped by region")
plt.grid()
plt.legend()

### Bokeh interactive plot

In [ ]:
src = ColumnDataSource(df_cumulative_region)
src.data

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool

colors = ['red', 'green', 'blue', 'grey', 'purple', 'black', 'orange']
# output to static HTML file
output_file("regions_cumulative.html")


# create a new plot with a title and axis labels
p2 = figure(title="Total deaths from Covid-19 in UK grouped by region", x_axis_label='Date',
           y_axis_label='Total deaths', x_axis_type="datetime")

for i, region in enumerate(regions):
    src = ColumnDataSource({'x':df_cumulative_region.index,
                           'y':df_cumulative_region[region]})
    p2.line(x='x', y='y', source=src, line_width=1, line_color=colors[i], legend=region)
    p2.circle(x='x', y='y', source=src, fill_color=colors[i], line_color=colors[i], size=6, legend=region)
    
    hover = HoverTool(tooltips =[
     ('region',region),('(Date, Total deaths)','(@x{%F}, @y)')],
                     formatters={'x': 'datetime'})
                     
    p2.add_tools(hover)

p2.legend.location = "top_left"

show(p2)

### Trust
Finally lets plot the trusts. I will try to plot these using some sort of map plot

In [ ]:
data_trusts